# RAG (Retrieval-Augmented Generation) Introduction

🚀 **Welcome to the RAG Tutorial!**

This notebook will guide you through building a complete RAG system from scratch. You'll learn:

- What RAG is and why it's important
- How to build a document processing pipeline
- Vector databases and similarity search
- Combining retrieval with generation

## What is RAG?

RAG (Retrieval-Augmented Generation) is a technique that combines:
- **Retrieval**: Finding relevant information from a knowledge base
- **Generation**: Using that information to generate informed responses

This approach allows AI systems to access and use specific knowledge while generating responses.


In [ ]:
# Install required packages (run this cell first)
!pip install sentence-transformers chromadb langchain matplotlib plotly

# Import required libraries
import sys
import os
sys.path.append('..')

from examples.rag.simple_rag import SimpleRAG, RAGConfig
from datasets.sample_data import generate_ai_documents
import matplotlib.pyplot as plt
import numpy as np

## Step 1: Initialize the RAG System

Let's start by creating and configuring our RAG system:

In [ ]:
# Create RAG configuration
config = RAGConfig(
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    chunk_size=500,
    chunk_overlap=100,
    top_k=3,
    collection_name="notebook_demo",
    persist_directory="./notebook_chroma_db"
)

# Initialize RAG system
rag = SimpleRAG(config)
rag.initialize()

print("✅ RAG system initialized successfully!")

## Step 2: Load Sample Documents

Let's load some sample AI-related documents to work with:

In [ ]:
# Generate sample documents
documents_data = generate_ai_documents()

# Extract content and metadata
documents = [doc['content'] for doc in documents_data]
metadata = [{'title': doc['title'], 'category': doc['category'], 'tags': doc['tags']} 
           for doc in documents_data]

print(f"📚 Loaded {len(documents)} documents:")
for i, doc_data in enumerate(documents_data):
    print(f"{i+1}. {doc_data['title']} (Category: {doc_data['category']})")

## Step 3: Add Documents to the RAG System

Now let's process and add these documents to our vector database:

In [ ]:
# Add documents to the RAG system
rag.add_documents(documents, metadata)

print(f"✅ Successfully added {len(documents)} documents to the RAG system!")
print(f"📊 Total chunks in database: {rag.collection.count()}")

## Step 4: Test RAG Queries

Let's test our RAG system with various queries:

In [ ]:
# Define test queries
test_queries = [
    "What is machine learning?",
    "How does deep learning work?",
    "What are the applications of computer vision?",
    "Explain reinforcement learning",
    "What is generative AI?"
]

# Test each query
for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"🔍 Query {i}: {query}")
    print(f"{'='*60}")
    
    # Search for relevant documents
    results = rag.search(query, top_k=2)
    
    print(f"\n📋 Found {len(results)} relevant chunks:")
    for j, result in enumerate(results, 1):
        print(f"\n{j}. Similarity Score: {1 - result.get('distance', 0):.3f}")
        print(f"   Content Preview: {result['document'][:200]}...")
        print(f"   Metadata: {result['metadata']}")
    
    # Generate response
    context_docs = [result['document'] for result in results]
    response = rag.generate_response(query, context_docs)
    
    print(f"\n🤖 Generated Response:")
    print(response)

## Step 5: Analyze Search Results

Let's visualize how well our queries match the documents:

In [ ]:
# Analyze query performance
query_analysis = []

for query in test_queries:
    results = rag.search(query, top_k=5)
    similarities = [1 - result.get('distance', 0) for result in results]
    query_analysis.append({
        'query': query,
        'max_similarity': max(similarities) if similarities else 0,
        'avg_similarity': np.mean(similarities) if similarities else 0,
        'num_results': len(results)
    })

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot max similarities
queries_short = [q[:30] + '...' if len(q) > 30 else q for q in test_queries]
max_sims = [analysis['max_similarity'] for analysis in query_analysis]
avg_sims = [analysis['avg_similarity'] for analysis in query_analysis]

ax1.bar(range(len(queries_short)), max_sims, alpha=0.7, label='Max Similarity')
ax1.bar(range(len(queries_short)), avg_sims, alpha=0.7, label='Avg Similarity')
ax1.set_xlabel('Queries')
ax1.set_ylabel('Similarity Score')
ax1.set_title('Query-Document Similarity Scores')
ax1.set_xticks(range(len(queries_short)))
ax1.set_xticklabels(queries_short, rotation=45, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot number of results
num_results = [analysis['num_results'] for analysis in query_analysis]
ax2.bar(range(len(queries_short)), num_results, color='green', alpha=0.7)
ax2.set_xlabel('Queries')
ax2.set_ylabel('Number of Results')
ax2.set_title('Number of Retrieved Documents')
ax2.set_xticks(range(len(queries_short)))
ax2.set_xticklabels(queries_short, rotation=45, ha='right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Display analysis summary
print("\n📊 Query Analysis Summary:")
for analysis in query_analysis:
    print(f"Query: {analysis['query'][:50]}...")
    print(f"  Max Similarity: {analysis['max_similarity']:.3f}")
    print(f"  Avg Similarity: {analysis['avg_similarity']:.3f}")
    print(f"  Results Found: {analysis['num_results']}\n")

## Step 6: Interactive Query Interface

Try your own queries with this interactive interface:

In [ ]:
def interactive_query():
    """
    Interactive query function for testing the RAG system
    """
    print("🎯 Interactive RAG Query Interface")
    print("Type 'quit' to exit\n")
    
    while True:
        try:
            # Get user query
            query = input("\n🔍 Enter your query: ").strip()
            
            if query.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye!")
                break
            
            if not query:
                print("⚠️ Please enter a valid query.")
                continue
            
            # Search and generate response
            print(f"\n🔍 Searching for: '{query}'")
            results = rag.search(query, top_k=3)
            
            if not results:
                print("❌ No relevant documents found.")
                continue
            
            print(f"\n📋 Found {len(results)} relevant chunks:")
            for i, result in enumerate(results, 1):
                similarity = 1 - result.get('distance', 0)
                print(f"{i}. Similarity: {similarity:.3f} | {result['document'][:100]}...")
            
            # Generate response
            context_docs = [result['document'] for result in results]
            response = rag.generate_response(query, context_docs)
            
            print(f"\n🤖 Response:")
            print("="*60)
            print(response)
            print("="*60)
            
        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

# Uncomment the line below to start the interactive interface
# interactive_query()

print("💡 Uncomment the last line in this cell to start the interactive interface!")

## Step 7: Advanced Features

Let's explore some advanced RAG features:

In [ ]:
# 1. Query with different top_k values
print("🔍 Testing different top_k values:")
query = "What is deep learning?"

for k in [1, 3, 5]:
    results = rag.search(query, top_k=k)
    print(f"\nTop-{k} results:")
    for i, result in enumerate(results, 1):
        similarity = 1 - result.get('distance', 0)
        print(f"  {i}. Similarity: {similarity:.3f}")

# 2. Analyze document categories
print("\n\n📊 Document category analysis:")
categories = {}
for doc_data in documents_data:
    category = doc_data['category']
    categories[category] = categories.get(category, 0) + 1

for category, count in categories.items():
    print(f"  {category}: {count} documents")

# 3. Test category-specific queries
print("\n\n🎯 Category-specific query analysis:")
category_queries = {
    "basics": "What are the fundamentals of AI?",
    "advanced": "How do advanced AI techniques work?",
    "applications": "What are practical AI applications?"
}

for category, query in category_queries.items():
    print(f"\n{category.upper()} Query: {query}")
    results = rag.search(query, top_k=2)
    
    for result in results:
        doc_category = result['metadata'].get('category', 'unknown')
        similarity = 1 - result.get('distance', 0)
        print(f"  Found in '{doc_category}' category (similarity: {similarity:.3f})")

## Summary and Next Steps

🎉 **Congratulations!** You've successfully built and tested a complete RAG system!

### What you learned:
1. **RAG Architecture**: Understanding retrieval + generation
2. **Document Processing**: Chunking and embedding
3. **Vector Search**: Finding relevant information
4. **Response Generation**: Creating informed answers
5. **Performance Analysis**: Evaluating search quality

### Next Steps:
1. **Try the multimodal notebook**: Work with images and text together
2. **Experiment with fine-tuning**: Customize models for your domain
3. **Build a web app**: Create user-friendly interfaces
4. **Scale up**: Handle larger document collections

### Resources:
- 📖 [Documentation](../docs/en/README.md)
- 🔬 [Advanced Examples](../examples/)
- 🛠️ [API Server](../automation/api_server.py)

Happy learning! 🚀